# 将原始bdf文件根据标签拆成样本，降采样到250Hz，并将每个样本保存为csv文件

In [1]:
import mne
import os
import pyedflib
import numpy as np
import pandas as pd

In [2]:
def readbdfdata(path):  # 读入原始数据文件夹路径
    # 文件夹下需包含data.bdf和evt.bdf两个文件，否则会报错
    path_data = path + 'data.bdf'
    path_evt = path + 'evt.bdf'
    if os.path.exists(path_data):
        pass
    else:
        raise NameError('In the path\'' + path_data +'\'No data.bdf file exists ！')
    if os.path.exists(path_evt):
        pass
    else:
        raise NameError('In the path\'' + path_evt +'\'No data.bdf file exists ！')
    annotation_info = pyedflib.EdfReader(path_evt) # 读入标签文件
    annotation = annotation_info.readAnnotations()      # 读入标签 打标签的时刻和标签的种类
    raw = mne.io.read_raw_edf(path_data,stim_channel=None,preload=True) # 读入数据文件
    # print('通道名称为： \n {0}'.format(np.transpose(raw.ch_names)))
    sfreq = raw.info['sfreq']
    events = np.transpose(np.asarray([annotation[0]*sfreq,annotation[1],annotation[2] ],dtype=int)) # 事件包括三列（打标签位置、0、标签类型）
    # raw.info['events'] = events.tolist()     # 增加events数组到info中（自己加的）
    # mne.utils.logger.info("%s events found" % len(events))
    # mne.utils.logger.info("Event IDs: {0}".format(np.unique(events[:, 2])))

    return _add_events_to_stim_channel(raw,events)


def _add_events_to_stim_channel(raw,events):    # 增加一个通道用来存储标签，打标签的时刻记录为1、2、3、4，其他时刻为0
    info = mne.create_info(['TRG'], raw.info['sfreq'], ['stim'])    # stim_channel的名字为'TRG'
    stim_data = np.zeros((1, raw.n_times))
    for evt in events:
        if evt is None:
            print('no events found')
            return
        stim_data[0, evt[0]] = evt[2]
    stim_raw = mne.io.RawArray(stim_data, info)
    raw.add_channels([stim_raw], force_update_info=True) # 增加标签通道到数据中
    return raw

In [3]:
bdfFath = r'/home/user/zhukai/data/trial/bdfFile'  # 原始bdf数据文件夹路径
fileFolder = os.listdir(bdfFath)
print(fileFolder)

['ZK180810', 'ZK180815_2', 'ZK180813_1', 'ZK180815_4', 'ZK180811', 'ZK180815_1', 'ZK180813_2', 'ZK180815_3', 'ZK180814_2', 'ZK180814_1', 'ZK180813_3']


In [4]:
channel_name = ['Fpz', 'Fp1', 'Fp2', 'AF3', 'AF4', 'AF7', 'AF8', 'Fz', 'F1', 'F2', 'F3', 'F4', 'F5', 'F6', 'F7',
                'F8', 'FCz', 'FC1', 'FC2', 'FC3', 'FC4', 'FC5', 'FC6', 'FT7', 'FT8', 'Cz', 'C1', 'C2', 'C3', 'C4',
                'C5', 'C6', 'T7', 'T8', 'CP1', 'CP2', 'CP3', 'CP4', 'CP5', 'CP6', 'TP7', 'TP8', 'Pz', 'P3', 'P4',
                'P5', 'P6', 'P7', 'P8', 'POz', 'PO3', 'PO4', 'PO5', 'PO6', 'PO7', 'PO8', 'Oz', 'O1', 'O2', 'ECG',
                'HEOR', 'HEOL', 'VEOU', 'VEOL', 'TRG'] # 所有通道

In [5]:
csvPath = r'/home/user/zhukai/data/trial/csvFile' # 保存数据文件夹根目录
for folder in fileFolder: # 对原始文件夹下的所有实验数据进行处理
    bdfFolderPath = bdfFath + r'/' + folder + r'/' # 单次实验数据的文件夹路径
    raw = readbdfdata(bdfFolderPath) # 将原始数据读成mne格式
    event = mne.find_events(raw, shortest_event=0, stim_channel='TRG') # 读入事件
    raw = raw.load_data()
    csvFolderPath = csvPath + r'/' + folder # 单次实验数据保存的文件夹
    folderExist = os.path.exists(csvFolderPath)
    if not folderExist:
        os.makedirs(csvFolderPath) # 创建文件夹
    for k in range(event.shape[0]): # 每个样本降采样后保存成.csv文件
        data = {}
        for i in range(65):
            data[channel_name[i]] = raw._data[i, event[k,0]-1000:event[k,0]+7000:4].tolist() #8000个点降采样到2000点，采样率降为250Hz
        data = pd.DataFrame(data)
        data.to_csv(csvFolderPath + r'/' + str(k+1) + '.csv')

Extracting EDF parameters from /home/user/zhukai/data/trial/bdfFile/ZK180810/data.bdf...
BDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 2958999  =      0.000 ...  2958.999 secs...
Creating RawArray with float64 data, n_channels=1, n_times=2959000
    Range : 0 ... 2958999 =      0.000 ...  2958.999 secs
Ready.
200 events found
Event IDs: [1 2 3 4]
Extracting EDF parameters from /home/user/zhukai/data/trial/bdfFile/ZK180815_2/data.bdf...
BDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 1481999  =      0.000 ...  1481.999 secs...
Creating RawArray with float64 data, n_channels=1, n_times=1482000
    Range : 0 ... 1481999 =      0.000 ...  1481.999 secs
Ready.
100 events found
Event IDs: [1 2 3 4]
Extracting EDF parameters from /home/user/zhukai/data/trial/bdfFile/ZK180813_1/data.bdf...
BDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 1469999  =